In [1]:
import os
import json
import shutil
import numpy as np
from tqdm import tqdm


In [2]:
!find . -iname '__MACOSX'

FIND: ������ʽ����ȷ


In [3]:
!find . -iname '.DS_Store'

FIND: ������ʽ����ȷ


In [4]:
!find . -iname '.ipynb_checkpoints'

FIND: ������ʽ����ȷ


In [5]:
!for i in `find . -iname '__MACOSX'`; do rm -rf $i;done

��ʱ��Ӧ�� i��


In [6]:
!for i in `find . -iname '.DS_Store'`; do rm -rf $i;done

��ʱ��Ӧ�� i��


In [7]:
!for i in `find . -iname '.ipynb_checkpoints'`; do rm -rf $i;done

��ʱ��Ӧ�� i��


In [8]:
!find . -iname '__MACOSX'

FIND: ������ʽ����ȷ


In [9]:
!find . -iname '.DS_Store'

FIND: ������ʽ����ȷ


In [10]:
!find . -iname '.ipynb_checkpoints'

FIND: ������ʽ����ȷ


In [ ]:
Dataset_root = '../../datasets/segmentation'  

In [12]:
classes = {
    '0': 0,
    '1': 1
    }  

In [13]:
os.chdir(Dataset_root)

In [14]:
with open('classes.txt', 'w', encoding='utf-8') as f:
    for each in list(classes.keys()):
        f.write(each + '\n')

In [15]:
os.mkdir('labels')
os.mkdir('labels/train')
os.mkdir('labels/val')

In [16]:
def process_single_json(labelme_path, save_folder='../../labels/train'):
    
    # 载入 labelme格式的 json 标注文件
    with open(labelme_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
        
    img_width = labelme['imageWidth']   # 图像宽度
    img_height = labelme['imageHeight'] # 图像高度
    
    # 生成 YOLO 格式的 txt 文件
    suffix = labelme_path.split('.')[-2]
    yolo_txt_path = suffix + '.txt'
    
    with open(yolo_txt_path, 'w', encoding='utf-8') as f:
        for each_ann in labelme['shapes']: # 遍历每个标注
            # 处理矩形和多边形标注
            if each_ann['shape_type'] in ['rectangle', 'polygon']:

                # 获取类别 ID
                bbox_class_id = classes[each_ann['label']]

                if each_ann['shape_type'] == 'rectangle':
                    # 左上角和右下角的 XY 像素坐标
                    bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
                    bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
                    bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
                    bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))

                    # 框中心点的 XY 像素坐标
                    bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
                    bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)

                    # 框宽度和高度
                    bbox_width = bbox_bottom_right_x - bbox_top_left_x
                    bbox_height = bbox_bottom_right_y - bbox_top_left_y

                    # 归一化坐标
                    bbox_center_x_norm = bbox_center_x / img_width
                    bbox_center_y_norm = bbox_center_y / img_height
                    bbox_width_norm = bbox_width / img_width
                    bbox_height_norm = bbox_height / img_height

                    # 生成 YOLO 格式字符串
                    bbox_yolo_str = '{} {:.4f} {:.4f} {:.4f} {:.4f}'.format(bbox_class_id, bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)

                # 多边形处理
                else:
                    # 多边形所有点的归一化坐标
                    points_norm = []
                    for (x, y) in each_ann['points']:
                        points_norm.append('{:.4f}'.format(x / img_width))
                        points_norm.append('{:.4f}'.format(y / img_height))
                    # 生成 YOLO 格式字符串（类别ID + 所有点坐标）
                    bbox_yolo_str = '{} {}'.format(bbox_class_id, ' '.join(points_norm))

                # 写入 txt 文件
                f.write(bbox_yolo_str + '\n')

    shutil.move(yolo_txt_path, save_folder)
    print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))

In [17]:
os.chdir('json/train')

In [18]:
save_folder = '../../labels/train'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

00-0001-20241212162530-00189.json --> 00-0001-20241212162530-00189.txt 转换完成
00-0001-20241212162530-00189_aug_1.json --> 00-0001-20241212162530-00189_aug_1.txt 转换完成
00-0001-20241212162530-00189_aug_3.json --> 00-0001-20241212162530-00189_aug_3.txt 转换完成
00-0002-20241212162530-00191_aug_1.json --> 00-0002-20241212162530-00191_aug_1.txt 转换完成
00-0002-20241212162530-00191_aug_2.json --> 00-0002-20241212162530-00191_aug_2.txt 转换完成
00-0002-20241212162530-00191_aug_3.json --> 00-0002-20241212162530-00191_aug_3.txt 转换完成
00-0002-20241212162530-00191_aug_4.json --> 00-0002-20241212162530-00191_aug_4.txt 转换完成
00-0003-20241226153258-00185.json --> 00-0003-20241226153258-00185.txt 转换完成
00-0003-20241226153258-00185_aug_1.json --> 00-0003-20241226153258-00185_aug_1.txt 转换完成
00-0003-20241226153258-00185_aug_3.json --> 00-0003-20241226153258-00185_aug_3.txt 转换完成
00-0004-20241226153258-00187.json --> 00-0004-20241226153258-00187.txt 转换完成
00-0004-20241226153258-00187_aug_2.json --> 00-0004-20241226153258-0

In [19]:
os.chdir('../../')

In [20]:
os.chdir('json/val')

In [21]:
save_folder = '../../labels/val'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

00-0001-20241212162530-00189_aug_2.json --> 00-0001-20241212162530-00189_aug_2.txt 转换完成
00-0001-20241212162530-00189_aug_4.json --> 00-0001-20241212162530-00189_aug_4.txt 转换完成
00-0002-20241212162530-00191.json --> 00-0002-20241212162530-00191.txt 转换完成
00-0003-20241226153258-00185_aug_2.json --> 00-0003-20241226153258-00185_aug_2.txt 转换完成
00-0003-20241226153258-00185_aug_4.json --> 00-0003-20241226153258-00185_aug_4.txt 转换完成
00-0004-20241226153258-00187_aug_1.json --> 00-0004-20241226153258-00187_aug_1.txt 转换完成
00-0004-20241226153258-00187_aug_4.json --> 00-0004-20241226153258-00187_aug_4.txt 转换完成
00-0005-20250407145627-00336.json --> 00-0005-20250407145627-00336.txt 转换完成
00-0005-20250407145627-00336_aug_2.json --> 00-0005-20250407145627-00336_aug_2.txt 转换完成
00-0005-20250407145627-00336_aug_3.json --> 00-0005-20250407145627-00336_aug_3.txt 转换完成
00-0006-20250303152800-00290.json --> 00-0006-20250303152800-00290.txt 转换完成
00-0006-20250303152800-00290_aug_1.json --> 00-0006-20250303152800-0

In [22]:
os.chdir('../../')

In [23]:
!rm -rf json

'rm' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
